# Neuroscience using `fastplotlib` :D

This notebook will build up a complex visualization using `fastplotlib`, in conjunction with other Python neuroscience packages, to exemplify how `fastplotlib` can be a powerful tool in analysis and visualization of neural data!

In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
import fastplotlib as fpl
import pynapple as nap
import numpy as np
from pathlib import Path
from ipywidgets.widgets import VBox, HBox, IntSlider, Layout
from mesmerize_core import *
from sidecar import Sidecar

### Opening behavior videos using **Lazy Loading** and visualizing them using `fastplotlib`.

#### **Lazy Loading**: strategy to conserve RAM by dynamically loading in frames for visualization only as needed 

Behavioral and neural data collected during experiments can be up to **terabytes** in size making it impossible to load all of the data into RAM at once for visualization and analysis. ***Lazy Loading*** allows us to bypass this memory space constraint without overburdening our resources!

In [5]:
# helper class from mesmerize_core for lazy loading
from mesmerize_core.arrays import LazyVideo

In [6]:
data = Path('/data/kushal/cortex-learning/2p-trial-exps/eaf1/behavior/session4/')

#### Get video paths for a single trial

In [7]:
trial_one = sorted(list(data.glob("*018.avi")))
trial_one

[PosixPath('/data/kushal/cortex-learning/2p-trial-exps/eaf1/behavior/session4/eaf1-s4_front_v018.avi'),
 PosixPath('/data/kushal/cortex-learning/2p-trial-exps/eaf1/behavior/session4/eaf1-s4_side_v018.avi')]

#### Make an `ImageWidget` to view the trial

In [8]:
warnings.simplefilter('ignore')

In [9]:
iw = fpl.ImageWidget(
    data=[LazyVideo(v) for v in trial_one], 
    names=["front", "side"],
    histogram_widget=False)

iw.show()

RFBOutputContext()

No config found!
No config found!


JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

### Visualizing results of running `CaImAn` via `mesmerize-core` on calcium imaging data.

In [10]:
set_parent_raw_data_path('/data/kushal/cortex-learning/2p-trial-exps/')

PosixPath('/data/kushal/cortex-learning/2p-trial-exps')

In [11]:
# load mesmerize batch that has already been run for this animal
df = load_batch(get_parent_raw_data_path().joinpath('eaf1/calcium/batch/batch.pickle'))
df.tail()

,algo,item_name,input_movie_path,params,outputs,comments,uuid,added_time,ran_time,algo_duration
22,cnmf,eaf1_session4,eaf1/calcium/session4/eaf4-s4-filtered-bilater...,"{'main': {'fr': 15, 'p': 1, 'nb': 2, 'merge_th...","{'success': False, 'traceback': 'Traceback (mo...",None,cc11149a-07d6-44ae-ba1a-f572744878c6,2023-01-23T18:10:34,2023-01-23T18:11:01,9.32 sec
23,cnmf,eaf1_session4,eaf1/calcium/session4/filtered-concat_session4...,"{'main': {'fr': 15, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': e1421f9b-d01a-4823-82...,None,e1421f9b-d01a-4823-8237-81a5ad7a952f,2023-01-23T19:08:59,2023-01-23T19:19:32,618.75 sec
24,cnmf,eaf1_session4,eaf1/calcium/session4/filtered-concat_session4...,"{'main': {'fr': 15, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 102e7ec8-bcdd-49ad-9a...,None,102e7ec8-bcdd-49ad-9a96-b4ef4041e70f,2023-01-23T20:17:17,2023-01-23T20:28:23,658.97 sec
25,cnmf,eaf1_session4,eaf1/calcium/session4/filtered-concat_session4...,"{'main': {'fr': 15, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 15118c77-e199-4baf-a3...,None,15118c77-e199-4baf-a35f-eb6b2c9c4a69,2023-01-23T20:36:59,2023-01-23T20:46:54,561.09 sec
26,cnmf,eaf1_session4,eaf1/calcium/session4/filtered-concat_session4...,"{'main': {'fr': 15, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 9b0f79f7-cbad-40d2-a1...,None,9b0f79f7-cbad-40d2-a1ef-f24fe6038cbb,2023-01-23T20:37:04,2023-01-23T21:02:48,542.55 sec


#### We are going to use the results of row 26

In [12]:
row_ix = 26

#### Get the motion corrected movie

In [13]:
raw = df.iloc[row_ix].caiman.get_input_movie()

#### Get contours and SNR components

In [14]:
# get the contours and center of masses using mesmerize_core
contours, coms = df.iloc[row_ix].cnmf.get_contours(component_indices="good", swap_dim=False)

# get the signal-to-noise ratio of each "good" component to color components 
snr_comps = df.iloc[row_ix].cnmf.get_output().estimates.SNR_comp

# get the good component_ixs
good_ixs = df.iloc[row_ix].cnmf.get_good_components()

# only get snr_comps of good_ixs
snr_comps = snr_comps[good_ixs]

np.log10(snr_comps)[:10]

array([ 0.24249106,  0.10373908, -0.13939034, -0.15015753,  0.08204876,
       -0.1908793 , -0.05140921, -0.17028186,  0.16581392,  0.44946934])

### Components with low SNR will **purple** and those with higher SNR ratio will be **yellow**

### Creating an interactive viualization to help with analysis

Will need a euclidean helper function to indentify which contours has been clicked on. We hope to soon include this, and other common callback functions, in our interactivity system :D

In [15]:
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    target.set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

#### Create a layout to display the raw neural activity and associated temporal data for each identified neuron

In [16]:
# create image widget for raw neural activity 
raw_iw = fpl.ImageWidget(
    data=[raw],
    names=["raw"]
)

# re-add our identified good components from before using the SNR mapping
contours_graphic = raw_iw.gridplot["raw"].add_line_collection(
                                                    data=contours, 
                                                    # use our snr components as cmap values
                                                    cmap_values=np.log10(snr_comps),
                                                    cmap="spring",
                                                    thickness=2,
                                                    name="contours")

# get temporal components
temporal = df.iloc[row_ix].cnmf.get_temporal(component_indices="good")

# temporal plot
plot_temporal = fpl.Plot(size=(600,100))
plot_temporal.add_line(temporal[0], colors="magenta")

# add a linear selector to temporal trace
plot_temporal.graphics[0].add_linear_selector()

# show temporal plot and mcorr/rcm plot in ipywidgets VBox 
sc = Sidecar()

with sc:
    display(VBox([raw_iw.show(), plot_temporal.show()]))

RFBOutputContext()

RFBOutputContext()

In [17]:
# link the linear selector to the image widget time slider
plot_temporal.selectors[0].add_ipywidget_handler(raw_iw.sliders["t"])

#### Interacitvity of `Graphics` using `link()` and callback functions

In [18]:
# link image to contours
image_graphic = raw_iw.gridplot["raw"]["image_widget_managed"]

image_graphic.link(
    "click",
    target=contours_graphic,
    feature="colors", 
    new_data="white", 
    callback=euclidean
)

# callback function to display correct temporal trace
def update_temporal(ev):
    # get data of selected ix
    data = temporal[ev.pick_info["collection-index"]]
    
    # add trace to plot 
    plot_temporal.graphics[0].data = data
    
    # autoscale the temporal plot
    plot_temporal.auto_scale()

# add event handler so that temporal trace is generated when contour is clicked on
contours_graphic[:].colors.add_event_handler(update_temporal)

# thickness of contour
contours_graphic.link("colors", target=contours_graphic, feature="thickness", new_data=5)

In [19]:
plot_temporal.close()
raw_iw.close()
sc.close()

### Syncing behavior and calcium data using `pynapple`

Having data input streams collected at different frame rates is a common problem. Here we will use `pynapple` in order to sync our two datastreams for future analysis.

In [20]:
import tifffile

#### We have pre-concatenated all of the behavior videos for this session to reduce the amount of boilerplate code

In [21]:
concat_behavior = tifffile.memmap("/home/clewis7/Desktop/sfn_data/preconcat.tiff")

In [22]:
concat_behavior.shape

(318850, 256, 336)

#### Recording Frame Rates:

In [23]:
behavior_fr = 500
calcium_fr = 15.2414

#### Create `pynapple` tensors for behavior and calcium data based on frame rate

In [24]:
# making a timestamp for the behavioral video
t_behavior = np.linspace(0, concat_behavior.shape[0] / behavior_fr, concat_behavior.shape[0])

In [25]:
t_behavior.shape

(318850,)

In [26]:
t_behavior[-1]

637.7

#### Use `pynapple` to create an indexable tensor of our behavioral data

In [27]:
tsd_video = nap.TsdTensor(t_behavior, concat_behavior)

In [28]:
# all of our time points and corresponding behavior frames mapped
tsd_video

Time (s)
-------------  -----------------
0.0            [[0 ... 9] ...]
0.002000006    [[0 ... 9] ...]
0.004000013    [[0 ... 9] ...]
0.006000019    [[15 ... 9] ...]
0.008000025    [[10 ... 9] ...]
...
637.691999975  [[29 ... 10] ...]
637.693999981  [[25 ... 10] ...]
637.695999987  [[20 ... 10] ...]
637.697999994  [[25 ... 10] ...]
637.7          [[25 ... 10] ...]
dtype: uint8, shape: (318850, 256, 336)

#### Do the same for our calcium data

In [29]:
# making a timestamp for the calcium video
t_calcium = np.linspace(0, raw.shape[0] / calcium_fr, raw.shape[0])

In [30]:
t_calcium.shape

(9153,)

In [31]:
t_calcium[-1]

600.5353838886191

#### Use `pynapple` to create an indexable tensor of our calcium data

In [32]:
tsd_calcium = nap.TsdTensor(t_calcium, raw)

#### Create a `GridPlot` for calcium and concatenated behavior videos

Here we are going to create a `GridPlot` and manually update the data in each subplot as we slide through time using our previously created `pynapple` tensors. 

The reason that we cannot use an `ImageWidget` here is because the framerate of our behavioral and calcium data is different. `ImageWidget` assumes that the data arrays in each subplot are all of the same framerate. 

In [47]:
nap_plot = fpl.GridPlot(shape=(1,2), names=[["raw", "behavior"]])

nap_plot["raw"].add_image(data=raw[0], name="raw_frame")
nap_plot["behavior"].add_image(data=tsd_video[0], cmap="gray")

RFBOutputContext()

<weakproxy at 0x7f8fd83278d0 to ImageGraphic at 0x7f8fd8018050>

#### Create slider that updates behavior and calcium data so they are aligned

In [49]:
# This time will be in milliseconds
synced_time = IntSlider(min=0, max=22*19 * 1_000, description="ms", layout=Layout(width="500"))

def update_time(change):
    # get the index of synced slider
    time_ms = change["new"]
    # get the corresponding calcium frame from the pynapple tensor
    frame_raw = tsd_calcium.get(time_ms, time_units="ms")
    # update the data in the plot
    nap_plot["raw"].graphics[0].data = frame_raw
    # get the corresponding behavior frame from the pynapple tensor
    frame_behavior = tsd_video.get(time_ms, time_units="ms")
    # update the data in the plot
    nap_plot["behavior"].graphics[0].data = frame_behavior

synced_time.observe(update_time, "value")

#### Re-create temporal plot and link to click events

In [50]:
# linear interpolation wrapper for temporal traces to expand the calcium timepoints to behavior
def interpolate(data: np.ndarray, new_size: int):
    """
    1D interpolation
    
    data is a 1d numpy array
    new size is the size to interpolate to
    """
    x = np.arange(0, new_size)
    xp = np.linspace(0, new_size, data.size)
    
    # interpolate to new size
    return np.interp(x, xp, fp=data)

In [51]:
# temporal plot
plot_temporal = fpl.Plot(size=(600,100))

interpolated_temporal = interpolate(data=temporal[0], new_size=t_behavior.shape[0])
plot_temporal.add_line(interpolated_temporal, colors="magenta")

# add a linear selector to temporal trace
plot_temporal.graphics[0].add_linear_selector()

# re-add our identified good components from before using the SNR mapping
contours_graphic = nap_plot["raw"].add_line_collection(
                                                    data=contours, 
                                                    # use our snr components as cmap values
                                                    cmap_values=np.log10(snr_comps),
                                                    cmap="spring",
                                                    thickness=2,
                                                    name="contours")

RFBOutputContext()

In [52]:
# link image to contours
image_graphic = nap_plot["raw"]["raw_frame"]

image_graphic.link(
    "click",
    target=contours_graphic,
    feature="colors", 
    new_data="white", 
    callback=euclidean
)

# callback function to display correct temporal trace
def update_temporal(ev):
    # get data of selected ix
    data = temporal[ev.pick_info["collection-index"]]
    
    # add trace to plot 
    interpolated_temporal = interpolate(data=data, new_size=t_behavior.shape[0])
    plot_temporal.graphics[0].data = interpolated_temporal
    
    # autoscale the temporal plot
    plot_temporal.auto_scale()

# add event handler so that temporal trace is generated when contour is clicked on
contours_graphic[:].colors.add_event_handler(update_temporal)

# thickness of contour
contours_graphic.link("colors", target=contours_graphic, feature="thickness", new_data=5)

def change_sync_slider_size(ev):
    synced_time.layout = Layout(width=str(ev.width))

# resize sliders based on temporal plot size
plot_temporal.renderer.add_event_handler(change_sync_slider_size, "resize")

<function __main__.change_sync_slider_size(ev)>

In [53]:
VBox([nap_plot.show(), plot_temporal.show(), synced_time])

In [54]:
# link the linear selector to the image widget time slider
plot_temporal.selectors[0].add_ipywidget_handler(synced_time)